In [ ]:
import random
import time
import json
import threading

class Player:
    """Creates the class for the Player to make user inputs"""
    def __init__(self, name, job):
        """Initializes name, and job of player"""
        self.name = name
        self.job = job
        self.inventory = []
        self.skills = self.skills()
        self.health = self.set_health()
        self.mana = self.set_mana()
        self.starting_equipment()
        self.equipped_weapon = None
        self.equipped_armor = None
        
    def set_health(self):
        """Sets the max health depending on what job is chosen"""
        if self.job == "Wizard":
            return 60
        elif self.job == "Warrior":
            return 100
        elif self.job == "Ranger":
            return 80
        
    def set_mana(self):
        """Sets the max mana depending on what job is chosen"""
        if self.job == "Wizard":
            return 100
        elif self.job == "Warrior":
            return 50
        elif self.job == "Ranger":
            return 80

    def skills(self):
        """Allows each job to use two skills associated with their job"""
        if self.job == "Wizard":
            return {"Fireball": (30, 38, 0, 25), "Magic Missile": (20, 40, 0, 30)}
        elif self.job == "Warrior":
            return {"Slash": (25, 33, 0, 20), "Stab": (20, 25, 3, 38)}
        elif self.job == "Ranger":
            return {"Triple Shot": (10, 30, 0, 15), "Deadshot": (30, 40, 0, 35)}
    
    def starting_equipment(self):
        """Gives starting equipment to the player based on their job"""
        if self.job == "Wizard":
            self.inventory.extend([Weapons("Oak Staff", "Reliable staff and perfect for the new wizard.", 12),
                                   Armor("Cloth Robe", "Offers little protection. Looks cool though.", 10),
                                   HP_Potions("Lesser Health Potion", "Heals 10 health.", 10),
                                   MP_Potions("Lesser Mana Potion", "Recover 10 mana.", 10)])
        elif self.job == "Warrior":
            self.inventory.extend([Weapons("Iron Sword", "A well crafted iron sword.", 10),
                                   Armor("Iron Armor", "This should protect me.", 25),
                                   HP_Potions("Lesser Health Potion", "Heals 10 health.", 10),
                                   MP_Potions("Lesser Mana Potion", "Recover 10 mana.", 10)])
        elif self.job == "Ranger":
            self.inventory.extend([Weapons("Oak Longbow", "A beautiful longbow made of luxurious Oak..", 12),
                                   Armor("Ranger's Robe", "Standard cloak given to rangers starting their journey.", 8),
                                   HP_Potions("Lesser Health Potion", "Heals 10 health.", 10),
                                   MP_Potions("Lesser Mana Potion", "Recover 10 mana.", 10)])
       
    def status(self):
        """Used to display necessary information regarding the player"""
        print(f"Job: {self.job}")
        print(f"{self.name}'s HP: {self.health}")
        print(f"{self.name}'s MP: {self.mana}")
        print("Inventory: ")
        for i, item in enumerate(self.inventory, start = 1):
            print(f"{i}. {item.name} | Description: {item.description}")
            if isinstance(item, Weapons):
                print(f"Damage: {item.damage}")
            elif isinstance(item, HP_Potions):
                print(f"Heal Amount: {item.heal_amount}")
            elif isinstance(item, MP_Potions):
                print(f"Recover Amount: {item.rec_amount}")
            elif isinstance(item, Armor):
                print(f"Defense: {item.defense}")
        print ("Skills: ")
        for skill, (min_damage, max_damage, _, mana_cost) in self.skills.items():
            print("-", skill, "Damage range: ", min_damage, " - ", max_damage, "Mana cost:", mana_cost)
            
    def add_item(self, item):
        """Allows players to pick up items into their inventory"""
        self.inventory.append(item)
        
    def take_damage(self, damage):
        """Allows players to receive damage"""
        self.health -= damage
        if self.health < 0:
            self.health = 0
        
    def use_mana(self, mana_cost):
        """Allows the use of mana for skills"""
        self.mana -= mana_cost
    
    def equip_gear(self):
        """Allows player to equip gear that they find in game"""
        choice = input("Enter the number associated with the item you want to equip (or 0 to cancel): ").strip()
        if choice.isdigit():
            index = int(choice) - 1
            if 0 <= index < len(self.inventory):
                item = self.inventory[index]
                if isinstance(item, Weapons):
                    self.equipped_weapon = item
                    print(f"{item.name} has been equipped!")
                elif isinstance(item, Armor):
                    self.equipped_armor = item
                    print(f"{item.name} has been equipped!")
                else:
                    print("You cannot equip this item.")
            elif index == -1:
                print("You chose not to equip anything.")
            else:
                print("Invalid choice.")
        else:
            print("Invalid input. Please enter a valid number.")
        
    def unequip_gear(self, item_index):
        """Allows users to unequip gear to equip new ones in game"""
        if item_index >= 0 and item_index < len(self.inventory): #conditional to check if item chosen is a valid number. if so item can be unequipped
            item = self.inventory.pop(item_index)
            print(f"{item.name} has been unequipped.")
            return item
        else:
            print("Invalid item index.")
            return None